In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib as mpl
import USERAUTH
import flickr_api
from flickr_api.api import flickr
import json

The inputs

In [2]:
city = gpd.read_file("arrondissements.shp") ## the shapefile of the place of interest
citycrs = 'epsg:27561' ## CRS of the place of interest
cityhex = gpd.read_file("paris_100m_hex.shp") ## hex grid shapefile
hexrad = '0.05' ## radius of hex in km as string

city = city.to_crs({'init': citycrs}) 


The functions

In [3]:
def centroid_coord(df): ## returns the input GeoDataFrame with added columns for lat and long in geodesic
    cent = df[['id','geometry']]
    cent = cent.to_crs({'init': 'epsg:4326'})
    cent['geometry'] = cent['geometry'].centroid
    cent['lat'] = cent['geometry'].apply(lambda p: p.y);
    cent['long'] = cent['geometry'].apply(lambda p: p.x);
    cent = cent[['id','lat','long']]
    return df.merge(cent, on='id') 

In [4]:
def hexpic(latd,long,rad): ## returns the number of pictures taken in integer form for a given lat, long, and radius,
    
    hex_pics_json = flickr.photos.search(privacy_filter = 1, accuracy = 16, lat=latd, lon=long, radius = rad, format= "json", nojsoncallback = 1)
    hexpics = json.loads(hex_pics_json)
    return int(hexpics["photos"]["total"])

In [5]:
def hydrate_hex(df, rad): ## returns the input GeoDataFrame with added column for number of picture. 'rad' is radius in km
    nrow = df.lat.count()
    df['pic'] = 0
    for x in range(0, nrow):
        df.loc[x,'pic'] = hexpic(df.loc[x,'lat'], df.loc[x,'long'], rad)
    return df

In [6]:
cityhex = centroid_coord(cityhex);
cityhex.head()

,fid,left,top,right,bottom,id,geometry,lat,long
0,1.0,602319.415266,132938.716906,602434.88532,132838.716906,38811,"POLYGON ((602319.415 132888.717, 602348.283 13...",48.896419,2.368927
1,2.0,602319.415266,133238.716906,602434.88532,133138.716906,38808,"POLYGON ((602319.415 133188.717, 602348.283 13...",48.899117,2.368929
2,3.0,602319.415266,133138.716906,602434.88532,133038.716906,38809,"POLYGON ((602319.415 133088.717, 602348.283 13...",48.898218,2.368928
3,4.0,602319.415266,133438.716906,602434.88532,133338.716906,38806,"POLYGON ((602319.415 133388.717, 602348.283 13...",48.900916,2.368930
4,5.0,602319.415266,133338.716906,602434.88532,133238.716906,38807,"POLYGON ((602319.415 133288.717, 602348.283 13...",48.900016,2.368929


In [ ]:
cityhex = hydrate_hex(cityhex,hexrad)
cityhex.head()